In [3]:
import pandas as pd

#import numpy as np
#from numpy import genfromtxt, savetxt

#import random
#from random import sample

#import matplotlib.pyplot as plt

#from sklearn.decomposition import PCA

import xgboost as xgb

#from sklearn import cross_validation
from sklearn.cross_validation import train_test_split, cross_val_score

#from sklearn.grid_search import GridSearchCV

#from sklearn.metrics import accuracy_score

import csv
import numpy as np

In [2]:
def read_numtrain(path):
    X_all_train=pd.read_csv(path, usecols=list(range(969)))
    print(X_all_train.columns)
    response=pd.read_csv(path, usecols=['Response'])
    print("######################### data ready ###########################")
    
    return X_all_train, response
    
    
    

In [8]:
def num_feature(X_train,response):
    column_name=X_train.columns
    
    grid = {'max_depth':10,
        #'booster': 'gbtree',
        'objective': 'binary:logistic',
        'subsample': 0.8,
        'colsample_bytree':0.8,
        'learning_rate': 0.1,
        'seed': 2016,
        'silent': 0,
        'gamma':0.05}

    clf1 = xgb.XGBClassifier()
    clf1.set_params(**grid)

    clf1.fit(X_train,response['Response'])
    
    importance=clf1.feature_importances_
    
    importance_s=sorted([(index,value) for index, value in enumerate(importance.tolist())],key=lambda x: x[1],reverse = True)
    
    important_feature_index=[index for (index,val) in importance_s if val>0.005]
    
    topindex=[index for (index,val) in importance_s][0:30]
    
    col_name = [column_name[index] for index in important_feature_index]
    
    top_name= [column_name[index] for index in topindex]
    
    #col_name = feature_names[importance> 0.005]
    print("TOP30", top_name)
    
    return col_name
    

In [9]:
def main():
    path='Y://Data//Kaggle//bosch//train_numeric//train_numeric.csv'
    X_train,response=read_numtrain(path)
    cols=num_feature(X_train,response)
    print(cols)
    

In [10]:
main()

Index(['Id', 'L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F8',
       'L0_S0_F10', 'L0_S0_F12', 'L0_S0_F14', 'L0_S0_F16',
       ...
       'L3_S50_F4243', 'L3_S50_F4245', 'L3_S50_F4247', 'L3_S50_F4249',
       'L3_S50_F4251', 'L3_S50_F4253', 'L3_S51_F4256', 'L3_S51_F4258',
       'L3_S51_F4260', 'L3_S51_F4262'],
      dtype='object', length=969)
######################### data ready ###########################
TOP30 ['Id', 'L3_S30_F3754', 'L3_S36_F3920', 'L3_S33_F3857', 'L3_S33_F3859', 'L3_S30_F3744', 'L0_S0_F20', 'L3_S33_F3865', 'L3_S30_F3759', 'L3_S30_F3749', 'L3_S30_F3809', 'L0_S1_F28', 'L3_S29_F3479', 'L3_S30_F3774', 'L3_S30_F3554', 'L3_S30_F3804', 'L3_S30_F3769', 'L3_S29_F3339', 'L3_S30_F3829', 'L3_S30_F3574', 'L3_S32_F3850', 'L1_S24_F1723', 'L3_S29_F3407', 'L3_S29_F3345', 'L3_S29_F3348', 'L3_S29_F3351', 'L3_S38_F3952', 'L3_S35_F3889', 'L3_S29_F3327', 'L3_S30_F3604']
['Id', 'L3_S30_F3754', 'L3_S36_F3920', 'L3_S33_F3857', 'L3_S33_F3859', 'L3_S30_F3744', 'L0_S0_F20', 'L3_S

In [32]:
#numeric min, max, mean, & std.
def pos(s):
    lst=s.tolist()
    n=len([ele for ele in lst if ele>0])
    
    return n

def neg(s):
    lst=s.tolist()
    n=len([ele for ele in lst if ele<=0])

    return n

def magic_feature_num():

    DATA_DIR = "Y://Data//Kaggle//bosch"

    ID_COLUMN = 'Id'
    TARGET_COLUMN = 'Response'

    SEED = 0
    CHUNKSIZE = 51500
    NROWS_TR = 1183747
    NROWS_TE = 1183748

    TRAIN_NUMERIC = "{0}/train_numeric.csv".format(DATA_DIR)

    TEST_NUMERIC = "{0}/test_numeric.csv".format(DATA_DIR)

    train = pd.read_csv(TRAIN_NUMERIC, usecols=[ID_COLUMN])
    test = pd.read_csv(TEST_NUMERIC, usecols=[ID_COLUMN])

    train["maxnum"] = -1
    train["minnum"] = -1
    train["mean"] = -1
    train["std"]=-1
    train["sum"]=-1
    train["nonan"]=-1
    train["pos"]=-1
    train["neg"]=-1
    
    test["maxnum"] = -1
    test["minnum"] = -1
    test["mean"] = -1
    test["std"]=-1
    test["sum"]=-1
    test["nonan"]=-1
    test["pos"]=-1
    test["neg"]=-1



    #date process
    #nrows = 0
    for tr in pd.read_csv(TRAIN_NUMERIC, chunksize=CHUNKSIZE):

        feats = np.setdiff1d(tr.columns, [ID_COLUMN, TARGET_COLUMN])
        minv = tr[feats].min(axis=1).values #min value of col
        maxv = tr[feats].max(axis=1).values
        mean = tr[feats].mean(axis=1).values
        summ = tr[feats].sum(axis=1).values 
        std = tr[feats].std(axis=1).values
        
        nonan= tr[feats].count(axis=1).tolist()
        poslst=tr[feats].apply(pos, axis=1).tolist()
        neglst=tr[feats].apply(neg, axis=1).tolist()
        


        train.loc[train.Id.isin(tr.Id), 'minnum'] = minv
        train.loc[train.Id.isin(tr.Id), 'maxnum'] = maxv
        train.loc[train.Id.isin(tr.Id), 'mean'] = mean
        train.loc[train.Id.isin(tr.Id), 'std'] = std
        train.loc[train.Id.isin(tr.Id), 'sum'] = summ
        train.loc[train.Id.isin(tr.Id), 'nonan'] = nonan
        train.loc[train.Id.isin(tr.Id), 'pos'] = poslst
        train.loc[train.Id.isin(tr.Id), 'neg'] = neglst
        
        print(1)

            
    for  te in pd.read_csv(TEST_NUMERIC, chunksize=CHUNKSIZE):

        feats = np.setdiff1d(te.columns, [ID_COLUMN])

        minv = te[feats].min(axis=1).values
        maxv = te[feats].max(axis=1).values
        mean = te[feats].mean(axis=1).values
        std = te[feats].std(axis=1).values
        summ = te[feats].sum(axis=1).values 
        
        nonan= te[feats].count(axis=1).tolist()
        
        poslst=te[feats].apply(pos, axis=1).tolist()
        neglst=te[feats].apply(neg, axis=1).tolist()

        test.loc[test.Id.isin(te.Id), 'minnum'] = minv
        test.loc[test.Id.isin(te.Id), 'maxnum'] = maxv
        test.loc[test.Id.isin(te.Id), 'mean'] = mean
        test.loc[test.Id.isin(te.Id), 'std'] = std
        test.loc[test.Id.isin(te.Id), 'sum'] = summ
        test.loc[test.Id.isin(te.Id), 'nonan'] = nonan
        
        test.loc[test.Id.isin(te.Id), 'pos'] = poslst
        test.loc[test.Id.isin(te.Id), 'neg'] = neglst        
        
        print(2)


#        nrows += CHUNKSIZE
#        if nrows >= NROWS:
#            break

    
    return train, test



In [35]:
s2

,Id,maxnum,minnum,mean,std,sum,nonan,pos,neg
0,1,0.520,-0.256,-0.006599,0.107915,-1.366,207,82,125
1,2,0.334,-0.480,-0.015459,0.110553,-3.401,220,83,137
2,3,0.520,-0.280,0.015310,0.120693,3.215,210,88,122
3,5,0.649,-0.611,-0.010326,0.148439,-1.776,172,68,104
4,8,0.520,-0.321,0.006967,0.117523,1.463,210,90,120
5,10,0.422,-0.480,-0.024095,0.107398,-4.578,190,62,128
6,12,0.520,-0.347,-0.008062,0.109202,-1.556,193,70,123
7,15,0.520,-0.338,0.008358,0.113000,1.588,190,79,111
8,17,0.489,-0.275,-0.001437,0.118009,-0.161,112,36,76
9,19,0.238,-0.480,-0.002924,0.095504,-0.614,210,93,117


In [36]:
s1.to_csv('Y://Data//Kaggle//bosch//num_desc_tr_1109.csv',index=False)
s2.to_csv('Y://Data//Kaggle//bosch//num_desc_te_1109.csv',index=False)

In [8]:
num_id=pd.read_csv('Y://Data//Kaggle//bosch//train_numeric.csv',usecols=['Id','Response'])
numid1=num_id[num_id['Response']==1]
idnum=numid1['Id']
s11=s1.loc[s1.Id.isin(idnum)] #subset whose label=1

In [40]:
s11

,Id,maxnum,minnum,mean,std,sum,nonan
539,1053,0.520,-0.356,0.018500,0.112101,2.960,162
633,1250,0.520,-0.268,-0.011113,0.109361,-1.967,179
681,1350,0.648,-0.308,-0.004328,0.125275,-0.883,206
887,1793,0.343,-0.480,-0.001295,0.106409,-0.246,192
1152,2347,0.395,-0.235,0.017370,0.135580,0.938,56
1731,3527,0.520,-0.458,0.009850,0.125942,1.576,162
1759,3585,0.906,-0.485,0.043139,0.179800,7.161,168
2028,4104,0.520,-0.326,-0.007806,0.136358,-1.249,162
2469,4978,0.520,-0.510,0.000756,0.116081,0.152,203
2479,4998,0.298,-0.480,-0.000595,0.107936,-0.113,192


In [45]:
"%0.4f" % 3

'3.0000'

In [51]:
#self add feature for date data

def date_feature():

    DATA_DIR = "Y://Data//Kaggle//bosch"

    ID_COLUMN = 'Id'
    TARGET_COLUMN = 'Response'

    SEED = 0
    CHUNKSIZE = 51500
    NROWS_TR = 1183747
    NROWS_TE = 1183748

    TRAIN_NUMERIC = "{0}/train_date.csv".format(DATA_DIR)

    TEST_NUMERIC = "{0}/test_date.csv".format(DATA_DIR)

    train = pd.read_csv(TRAIN_NUMERIC, usecols=[ID_COLUMN])
    test = pd.read_csv(TEST_NUMERIC, usecols=[ID_COLUMN])

    train["nonan"]=-1
    train['mean']=-1

    test["nonan"]=-1
    test['mean']=-1



    #date process
    #nrows = 0
    for tr in pd.read_csv(TRAIN_NUMERIC, chunksize=CHUNKSIZE):

        feats = np.setdiff1d(tr.columns, [ID_COLUMN])
        

        
        nonan= tr.count(axis=1).tolist()
        meann= tr[feats].mean(axis=1).values
    


        train.loc[train.Id.isin(tr.Id), 'nonan'] = nonan
        train.loc[train.Id.isin(tr.Id), 'mean'] = meann
        
        print(1)



        #nrows += CHUNKSIZE

        #if nrows >= NROWS:
            #break 
            
    for  te in pd.read_csv(TEST_NUMERIC, chunksize=CHUNKSIZE):

        feats = np.setdiff1d(te.columns, [ID_COLUMN])

        
        nonan= te.count(axis=1).tolist()
        meann= te[feats].mean(axis=1).values


        test.loc[test.Id.isin(te.Id), 'nonan'] = nonan
        test.loc[test.Id.isin(te.Id), 'mean'] = meann
        print(2)


#        nrows += CHUNKSIZE
#        if nrows >= NROWS:
#            break

    
    return train, test



In [56]:
s1.to_csv('Y://Data//Kaggle//bosch//date_desc_tr.csv',index=False)
s2.to_csv('Y://Data//Kaggle//bosch//date_desc_te.csv',index=False)